## Tokenizer for Fine-Tuning Language Model 💻

In [1]:
# XLM Roberta Base - Tokenizer Download From HuggingFace
from transformers import XLMRobertaTokenizer
xlm_roberta_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

/Users/waterondaway/Laboratory/medicine-scan-lm/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# PythaiNLP - Tokenizer Download From Library
from pythainlp.tokenize import word_tokenize as pythainlp_tokenizer

### Data Preprocessing ⚙️

In [ ]:
# Import dataset to tokenizer
import pandas as pd
dataset = pd.read_csv('../data/dataset.csv')

In [8]:
# Display sample of dataset
dataset.head(3)

,PATIENT_NAME,PATIENT_ID,PATIENT_BIRTHDATE,DRUG_NAME,DOSAGE,FORM,DRUG_REG_NO,MFG_DATE,EXP_DATE,WARNINGS,INDICATIONS,USAGE_INSTRUCTIONS
0,วรรณรัตน์ เพียยา,7383942134581,12 มิถุนายน 2489,ไอบูโพรเฟน,12.5mg,เม็ด,REG026,2024-10-15,2026-08-18,ควรใช้ตามคำแนะนำของแพทย์,ลดไข้ในผู้ป่วยไข้สูง,เขย่าขวดให้เข้ากันก่อนทุกครั้ง
1,ภานุพล นาฏคายี,5672192371808,12 สิงหาคม 2501,Diclofenac,500mg,เม็ด,REG063,2024-05-07,2028-01-26,ควรเก็บยาให้พ้นมือเด็ก,ลดอาการเจ็บปวดหลังผ่าตัด,รับประทาน 2 เม็ดก่อนนอน
2,พิมพกานต์ นามเสวตร,5737702286578,10 เมษายน 2481,Aspirin,5mg,tablet,REG011,2025-02-16,2027-12-07,ห้ามเกินขนาดที่แนะนำ,บรรเทาอาการจากการติดเชื้อทางเดินอาหาร,รับประทานวันละ 1 เม็ดหลังมื้อกลางวัน


#### Automate tag labels for XLM Roberta Tokenizer 😈

In [9]:
# Function to automate tag labels for each data
def automate_ner_tags_xlm_roberta_tokenizer(sentence):
    patient_name_tokens = xlm_roberta_tokenizer.tokenize(str(sentence["PATIENT_NAME"]))
    patient_id_tokens = xlm_roberta_tokenizer.tokenize(str(sentence["PATIENT_ID"]))
    patient_birthdate_tokens = xlm_roberta_tokenizer.tokenize(str(sentence["PATIENT_BIRTHDATE"]))
    drug_name_tokens = xlm_roberta_tokenizer.tokenize(sentence["DRUG_NAME"])
    dosage_tokens = xlm_roberta_tokenizer.tokenize(sentence["DOSAGE"])
    form_tokens = xlm_roberta_tokenizer.tokenize(sentence["FORM"])
    drug_reg_no_tokens = xlm_roberta_tokenizer.tokenize(sentence["DRUG_REG_NO"])
    mfg_date_tokens = xlm_roberta_tokenizer.tokenize(sentence["MFG_DATE"])
    exp_date_tokens = xlm_roberta_tokenizer.tokenize(sentence["EXP_DATE"])
    warnings_tokens = xlm_roberta_tokenizer.tokenize(sentence["WARNINGS"])
    indications_tokens = xlm_roberta_tokenizer.tokenize(sentence["INDICATIONS"])
    usage_instructions_tokens = xlm_roberta_tokenizer.tokenize(sentence["USAGE_INSTRUCTIONS"])
    
    patient_name_labels = ["O"] * (len(patient_name_tokens)-1)
    patient_id_labels = ["O"] * (len(patient_id_tokens)-1)
    patient_birthdate_labels = ["O"] * (len(patient_birthdate_tokens)-1)
    drug_name_labels = ["B-DRUG_NAME"] + ["I-DRUG_NAME"] * (len(drug_name_tokens)-1)
    dosage_labels = ["B-DOSAGE"] + ["I-DOSAGE"] * (len(dosage_tokens) - 1)
    form_labels = ["B-FORM"] + ["I-FORM"] * (len(form_tokens) - 1)
    drug_reg_no_labels = ["B-DRUG_REG_NO"] + ["I-DRUG_REG_NO"] * (len(drug_reg_no_tokens) - 1)
    mfg_date_labels = ["B-MFG_DATE"] + ["I-MFG_DATE"] * (len(mfg_date_tokens)-1)
    exp_date_labels = ["B-EXP_DATE"] + ["I-EXP_DATE"] * (len(exp_date_tokens)-1)
    warnings_labels = ["B-WARNINGS"] + ["I-WARNINGS"] * (len(warnings_tokens)-1)
    indications_labels = ["B-INDICATIONS"] + ["I-INDICATIONS"] * (len(indications_tokens)-1)
    usage_instructions_labels = ["B-USAGE_INSTRUCTIONS"] + ["I-USAGE_INSTRUCTIONS"] * (len(usage_instructions_tokens)-1)
    
    tokens = patient_name_tokens + patient_id_tokens + patient_birthdate_tokens + drug_name_tokens + dosage_tokens + form_tokens + drug_reg_no_tokens + mfg_date_tokens + exp_date_tokens + warnings_tokens + indications_tokens + usage_instructions_tokens
    ner_tags = patient_name_labels + patient_id_labels + patient_birthdate_labels + drug_name_labels + dosage_labels + form_labels + drug_reg_no_labels + mfg_date_labels + exp_date_labels + warnings_labels + indications_labels + usage_instructions_labels
    
    return {
        "tokens": tokens,
        "ner_tags": ner_tags
    }

In [14]:
# Calling function 
xlm_roberta_tokenizer_formatted_dataset = [automate_ner_tags_xlm_roberta_tokenizer(sentence) for index, sentence in dataset.iterrows()]

# Save with JSON File
import json
with open('../data/xlm_roberta_tokenizer_format.json', 'w', encoding='utf-8') as f:
    json.dump(xlm_roberta_tokenizer_formatted_dataset, f, ensure_ascii=False, indent=4)

#### Automate tag labels for PythaiNLP Tokenizer 😈

In [15]:
# Function to automate tag labels for each data
def automate_ner_tags_pythainlp_tokenizer(sentence, engine_setup):
    patient_name_tokens = pythainlp_tokenizer(sentence["PATIENT_NAME"], engine=engine_setup)
    patient_id_tokens = pythainlp_tokenizer(sentence["PATIENT_ID"], engine=engine_setup)
    patient_birthdate_tokens = pythainlp_tokenizer(sentence["PATIENT_BIRTHDATE"], engine=engine_setup)
    drug_name_tokens = pythainlp_tokenizer(sentence["DRUG_NAME"], engine=engine_setup)
    dosage_tokens = pythainlp_tokenizer(sentence["DOSAGE"], engine=engine_setup)
    form_tokens = pythainlp_tokenizer(sentence["FORM"], engine=engine_setup)
    drug_reg_no_tokens = pythainlp_tokenizer(sentence["DRUG_REG_NO"], engine=engine_setup)
    mfg_date_tokens = pythainlp_tokenizer(sentence["MFG_DATE"], engine=engine_setup)
    exp_date_tokens = pythainlp_tokenizer(sentence["EXP_DATE"], engine=engine_setup)
    warnings_tokens = pythainlp_tokenizer(sentence["WARNINGS"], engine=engine_setup)
    indications_tokens = pythainlp_tokenizer(sentence["INDICATIONS"], engine=engine_setup)
    usage_instructions_tokens = pythainlp_tokenizer(sentence["USAGE_INSTRUCTIONS"], engine=engine_setup)
    
    patient_name_labels = ["O"] * (len(patient_name_tokens)-1)
    patient_id_labels = ["O"] * (len(patient_id_tokens)-1)
    patient_birthdate_labels = ["O"] * (len(patient_birthdate_tokens)-1)
    drug_name_labels = ["B-DRUG_NAME"] + ["I-DRUG_NAME"] * (len(drug_name_tokens)-1)
    dosage_labels = ["B-DOSAGE"] + ["I-DOSAGE"] * (len(dosage_tokens) - 1)
    form_labels = ["B-FORM"] + ["I-FORM"] * (len(form_tokens) - 1)
    drug_reg_no_labels = ["B-DRUG_REG_NO"] + ["I-DRUG_REG_NO"] * (len(drug_reg_no_tokens) - 1)
    mfg_date_labels = ["B-MFG_DATE"] + ["I-MFG_DATE"] * (len(mfg_date_tokens)-1)
    exp_date_labels = ["B-EXP_DATE"] + ["I-EXP_DATE"] * (len(exp_date_tokens)-1)
    warnings_labels = ["B-WARNINGS"] + ["I-WARNINGS"] * (len(warnings_tokens)-1)
    indications_labels = ["B-INDICATIONS"] + ["I-INDICATIONS"] * (len(indications_tokens)-1)
    usage_instructions_labels = ["B-USAGE_INSTRUCTIONS"] + ["I-USAGE_INSTRUCTIONS"] * (len(usage_instructions_tokens)-1)
    
    tokens = patient_name_tokens + patient_id_tokens + patient_birthdate_tokens + drug_name_tokens + dosage_tokens + form_tokens + drug_reg_no_tokens + mfg_date_tokens + exp_date_tokens + warnings_tokens + indications_tokens + usage_instructions_tokens
    ner_tags = patient_name_labels + patient_id_labels + patient_birthdate_labels + drug_name_labels + dosage_labels + form_labels + drug_reg_no_labels + mfg_date_labels + exp_date_labels + warnings_labels + indications_labels + usage_instructions_labels
    
    return {
        "tokens": tokens,
        "ner_tags": ner_tags
    }

In [16]:
# Calling function 
pythainlp_tokenizer_formatted_dataset = [automate_ner_tags_pythainlp_tokenizer(sentence, 'longest') for index, sentence in dataset.iterrows()]

# Save with JSON File
import json
with open('../data/pythainlp_tokenizer_format.json', 'w', encoding='utf-8') as f:
    json.dump(pythainlp_tokenizer_formatted_dataset, f, ensure_ascii=False, indent=4)